## CSE 258 Web Mining and Recommender Systems
### Assignment 1 
**Shivani Bhakta** 

In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [5]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [8]:
allRatings = []
for l in readCSV("assignment1/train_Interactions.csv.gz"):
    allRatings.append(l)
# len(allRatings) # 200000

In [9]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))
    
#Note: for ratingsTrain, 1st col: user id, 2nd col: book id,
# 3rd col: rating 

In [10]:
##################################################
# Rating prediction (CSE258 only)                #
##################################################

In [11]:
trainRatings = [r[2] for r in ratingsTrain]
globalAverage = sum(trainRatings) * 1.0 / len(trainRatings)

In [12]:
validMSE = 0
for u,b,r in ratingsValid:
    se = (r - globalAverage)**2
    validMSE += se

validMSE /= len(ratingsValid)

print("Validation MSE (average only) = " + str(validMSE))

Validation MSE (average only) = 1.6802113179223874


In [13]:
betaU = {}
betaI = {}
for u in ratingsPerUser:
    betaU[u] = 0

for b in ratingsPerItem:
    betaI[b] = 0
alpha = globalAverage 
# Could initialize anywhere, this is a guess

In [14]:
alpha

3.6868052631578947

In [15]:
def iterate(lamb):
    newAlpha = 0
    for u,b,r in ratingsTrain:
        newAlpha += r - (betaU[u] + betaI[b])
    alpha = newAlpha / len(ratingsTrain)
    for u in ratingsPerUser:
        newBetaU = 0
        for b,r in ratingsPerUser[u]:
            newBetaU += r - (alpha + betaI[b])
        betaU[u] = newBetaU / (lamb + len(ratingsPerUser[u]))
    for b in ratingsPerItem:
        newBetaI = 0
        for u,r in ratingsPerItem[b]:
            newBetaI += r - (alpha + betaU[u])
        betaI[b] = newBetaI / (lamb + len(ratingsPerItem[b]))
    mse = 0
    for u,b,r in ratingsTrain:
        prediction = alpha + betaU[u] + betaI[b]
        mse += (r - prediction)**2
    regularizer = 0
    for u in betaU:
        regularizer += betaU[u]**2
    for b in betaI:
        regularizer += betaI[b]**2
    mse /= len(ratingsTrain)
    return mse, mse + lamb*regularizer

In [16]:
mse,objective = iterate(1)
newMSE,newObjective = iterate(1)
iterations = 2

In [17]:
while iterations < 10 or objective - newObjective > 0.0001:
    mse, objective = newMSE, newObjective
    newMSE, newObjective = iterate(1)
    iterations += 1
    print("Objective after "
        + str(iterations) + " iterations = " + str(newObjective))
    print("MSE after "
        + str(iterations) + " iterations = " + str(newMSE))

Objective after 3 iterations = 13115.640591454927
MSE after 3 iterations = 1.0546946984600858
Objective after 4 iterations = 13124.694330263232
MSE after 4 iterations = 1.054567743502505
Objective after 5 iterations = 13122.40568567617
MSE after 5 iterations = 1.054538428024114
Objective after 6 iterations = 13117.55582265247
MSE after 6 iterations = 1.0545272453633137
Objective after 7 iterations = 13112.275464784787
MSE after 7 iterations = 1.0545205501660955
Objective after 8 iterations = 13107.097417003808
MSE after 8 iterations = 1.0545152507460684
Objective after 9 iterations = 13102.15694106679
MSE after 9 iterations = 1.0545105256034721
Objective after 10 iterations = 13097.483282354282
MSE after 10 iterations = 1.0545061301488619
Objective after 11 iterations = 13093.07561251617
MSE after 11 iterations = 1.0545019823151736
Objective after 12 iterations = 13088.924297196943
MSE after 12 iterations = 1.054498049188854
Objective after 13 iterations = 13085.017240657753
MSE after 

Objective after 89 iterations = 13026.674940104496
MSE after 89 iterations = 1.0544210361049515
Objective after 90 iterations = 13026.660259607908
MSE after 90 iterations = 1.0544209117606178
Objective after 91 iterations = 13026.647054557925
MSE after 91 iterations = 1.0544207918951392
Objective after 92 iterations = 13026.635217342768
MSE after 92 iterations = 1.0544206763312192
Objective after 93 iterations = 13026.624647440403
MSE after 93 iterations = 1.0544205648995126
Objective after 94 iterations = 13026.615250971123
MSE after 94 iterations = 1.054420457438071
Objective after 95 iterations = 13026.60694027839
MSE after 95 iterations = 1.0544203537920764
Objective after 96 iterations = 13026.599633534444
MSE after 96 iterations = 1.0544202538133658
Objective after 97 iterations = 13026.593254370538
MSE after 97 iterations = 1.0544201573603038
Objective after 98 iterations = 13026.587731530637
MSE after 98 iterations = 1.0544200642972965
Objective after 99 iterations = 13026.5829

In [18]:
validMSE = 0
for u,b,r in ratingsValid:
    bu = 0
    bi = 0
    if u in betaU:
        bu = betaU[u]
    if b in betaI:
        bi = betaI[b]
    prediction = alpha + bu + bi
    validMSE += (r - prediction)**2

validMSE /= len(ratingsValid)
print("Validation MSE = " + str(validMSE))

Validation MSE = 1.4560187911380087


In [19]:
betaUs = [(betaU[u], u) for u in betaU]
betaIs = [(betaI[i], i) for i in betaI]
betaUs.sort()
betaIs.sort()

print("Maximum betaU = " + str(betaUs[-1][1]) + ' (' + str(betaUs[-1][0]) + ')')
print("Maximum betaI = " + str(betaIs[-1][1]) + ' (' + str(betaIs[-1][0]) + ')')
print("Minimum betaU = " + str(betaUs[0][1]) + ' (' + str(betaUs[0][0]) + ')')
print("Minimum betaI = " + str(betaIs[0][1]) + ' (' + str(betaIs[0][0]) + ')')

Maximum betaU = u19874911 (1.830043177768543)
Maximum betaI = b57808985 (1.5787383903515075)
Minimum betaU = u56426501 (-3.5547578823516117)
Minimum betaI = b00236726 (-2.2786063206642573)


In [20]:
# Better lambda...

iterations = 1
while iterations < 10 or objective - newObjective > 0.0001:
    mse, objective = newMSE, newObjective
    newMSE, newObjective = iterate(5)
    iterations += 1
    print("Objective after " + str(iterations) + " iterations = " + str(newObjective))
    print("MSE after " + str(iterations) + " iterations = " + str(newMSE))

Objective after 2 iterations = 26651.49272507437
MSE after 2 iterations = 1.1533560163859182
Objective after 3 iterations = 26127.872513753802
MSE after 3 iterations = 1.1555101451692789
Objective after 4 iterations = 26082.792859614645
MSE after 4 iterations = 1.155738973684075
Objective after 5 iterations = 26076.37162849408
MSE after 5 iterations = 1.1557710771682903
Objective after 6 iterations = 26074.781854419038
MSE after 6 iterations = 1.1557781466827532
Objective after 7 iterations = 26074.00744005284
MSE after 7 iterations = 1.155781138704707
Objective after 8 iterations = 26073.434582928654
MSE after 8 iterations = 1.1557832610369514
Objective after 9 iterations = 26072.951008500215
MSE after 9 iterations = 1.1557850812266992
Objective after 10 iterations = 26072.53056105207
MSE after 10 iterations = 1.1557867114719453
Objective after 11 iterations = 26072.16312166275
MSE after 11 iterations = 1.1557881805003798
Objective after 12 iterations = 26071.841878161627
MSE after 12

In [21]:
validMSE = 0
for u,b,r in ratingsValid:
    bu = 0
    bi = 0
    if u in betaU:
        bu = betaU[u]
    if b in betaI:
        bi = betaI[b]
    prediction = alpha + bu + bi
    validMSE += (r - prediction)**2

validMSE /= len(ratingsValid)
print("Validation MSE = " + str(validMSE))
lamb =  5.0

Validation MSE = 1.400379375841991


In [22]:
answers['Q11'] = (lamb, validMSE)

In [23]:
assertFloat(answers['Q11'][0])
assertFloat(answers['Q11'][1])

In [24]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("assignment1/pairs_Rating.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    bu = 0
    bi = 0
    if u in betaU:
        bu = betaU[u]
    if b in betaI:
        bi = betaI[b]
    _ = predictions.write(u + ',' + b + ',' + str(alpha + bu + bi) + '\n')

predictions.close()

In [33]:
##################################################
# Read prediction                                #
##################################################

In [34]:
# From baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("assignment1/train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

In [35]:
# Generate a negative set

userSet = set()
bookSet = set()
readSet = set()

for u,b,r in allRatings:
    userSet.add(u)
    bookSet.add(b)
    readSet.add((u,b))

lUserSet = list(userSet)
lBookSet = list(bookSet)

notRead = set()
for u,b,r in ratingsValid:
    #u = random.choice(lUserSet)
    b = random.choice(lBookSet)
    while ((u,b) in readSet or (u,b) in notRead):
        b = random.choice(lBookSet)
    notRead.add((u,b))

readValid = set()
for u,b,r in ratingsValid:
    readValid.add((u,b))

In [36]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom > 0:
        return numer/denom
    return 0

In [37]:
### Question 1

In [38]:
# Evaluate baseline strategy

correct = 0
p0, p1 = 0,0
for (label,sample) in [(1, readValid), (0, notRead)]:
    for (u,b) in sample:
        pred = 0
        if b in return1:
            pred = 1
        if pred == label:
            correct += 1

In [39]:
acc1 = correct / (len(readValid) + len(notRead))
acc1

0.70805

In [40]:
answers['Q1'] = acc1

In [41]:
assertFloat(answers['Q1'])

In [42]:
### Question 2

In [43]:
# Improved strategy

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > 1.5 * totalRead/2: break

In [44]:
# Evaluate baseline strategy

correct = 0
p0, p1 = 0,0
for (label,sample) in [(1, readValid), (0, notRead)]:
    for (u,b) in sample:
        pred = 0
        if b in return1:
            pred = 1
        if pred == label:
            correct += 1

In [45]:
threshold = 1.5 * totalRead/2
acc2 = correct / (len(readValid) + len(notRead))
threshold, acc2

(150000.0, 0.7507)

In [46]:
answers['Q2'] = [threshold, acc2]

In [47]:
assertFloat(answers['Q2'][0])
assertFloat(answers['Q2'][1])

In [48]:
### Question 3/4

In [49]:
# Slow implementation, could easily be improved

correct = 0
for (label,sample) in [(1, readValid), (0, notRead)]:
    for (u,b) in sample:
        maxSim = 0
        users = set(ratingsPerItem[b])
        for b2,_ in ratingsPerUser[u]:
            sim = Jaccard(users,set(ratingsPerItem[b2]))
            if sim > maxSim:
                maxSim = sim
        pred = 0
        if maxSim > 0.013 or len(ratingsPerItem[b]) > 40:
            pred = 1
        if pred == label:
            correct += 1

In [50]:
acc3 = correct / (len(readValid) + len(notRead))
acc4 = correct / (len(readValid) + len(notRead))
acc3

0.74355

In [51]:
answers['Q3'] = acc3
answers['Q4'] = acc4

In [52]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [53]:
predictions = open("predictions_Read.csv", 'w')
for l in open("assignment1/pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    maxSim = 0
    users = set(ratingsPerItem[b])
    for b2,_ in ratingsPerUser[u]:
        sim = Jaccard(users,set(ratingsPerItem[b2]))
        if sim > maxSim:
            maxSim = sim
    pred = 0
    if maxSim > 0.013 or len(ratingsPerItem[b]) > 40:
        pred = 1
    _ = predictions.write(u + ',' + b + ',' + str(pred) + '\n')

predictions.close()

In [54]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [55]:
assert type(answers['Q5']) == str

In [56]:
##################################################
# Category prediction (CSE158 only)              #
##################################################

In [57]:
### Question 6

In [58]:
data = []

for d in readGz("assignment1/train_Category.json.gz"):
    data.append(d)

In [59]:
data[0]

{'user_id': 'u75242413',
 'review_id': 'r45843137',
 'rating': 4,
 'review_text': "a clever book with a deeply troubling premise and an intriguing protagonist. Thompson's clean, sparse prose style kept each page feeling light even as some rather heavy existential questions dropped upon them. I enjoyed it. \n and that cover design is boom-pow gorgeous.",
 'n_votes': 1,
 'genre': 'mystery_thriller_crime',
 'genreID': 3}

In [60]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [61]:
counts[:10]

[(739469, 'the'),
 (447352, 'and'),
 (393557, 'a'),
 (373089, 'to'),
 (364764, 'i'),
 (324675, 'of'),
 (218882, 'is'),
 (212488, 'in'),
 (204960, 'it'),
 (193791, 'this')]

In [62]:
answers['Q6'] = counts[:10]

In [63]:
assert [type(x[0]) for x in answers['Q6']] == [int]*10
assert [type(x[1]) for x in answers['Q6']] == [str]*10

In [64]:
### Question 7

In [65]:
NW = 1000 # dictionary size

In [66]:
words = [x[1] for x in counts[:NW]]

In [67]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    for w in r.split():
        if w in wordSet:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data]
y = [d['genreID'] for d in data]

In [68]:
Xtrain = X[:9*len(X)//10]
ytrain = y[:9*len(y)//10]
Xvalid = X[9*len(X)//10:]
yvalid = y[9*len(y)//10:]

In [69]:
mod = linear_model.LogisticRegression(C=1)

In [70]:
mod.fit(Xtrain, ytrain)

LogisticRegression(C=1)

In [71]:
pred = mod.predict(Xvalid)

In [72]:
correct = pred == yvalid

In [73]:
sum(correct) / len(correct)

0.6411

In [74]:
acc7 =  float(sum(correct) / len(correct))

In [75]:
answers['Q7'] = acc7

In [76]:
assertFloat(answers['Q7'])

In [77]:
### Question 8

In [78]:
# Same as previous solution, with larger dictionary

NW = 2000 # dictionary size

words = [x[1] for x in counts[:NW]]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    for w in r.split():
        if w in wordSet:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data]
y = [d['genreID'] for d in data]

Xtrain = X[:9*len(X)//10]
ytrain = y[:9*len(y)//10]
Xvalid = X[9*len(X)//10:]
yvalid = y[9*len(y)//10:]

mod = linear_model.LogisticRegression(C=1)
mod.fit(Xtrain, ytrain)

pred = mod.predict(Xvalid)
correct = pred == yvalid
sum(correct) / len(correct)

0.6905

In [79]:
acc8 = float(sum(correct) / len(correct))

In [80]:
answers['Q8'] = acc8

In [81]:
assertFloat(answers['Q8'])

In [82]:
# Run on test set

In [83]:
data_test = []

for d in readGz("assignment1/test_Category.json.gz"):
    data_test.append(d)

In [84]:
Xtest = [feature(d) for d in data_test]
pred_test = mod.predict(Xtest)

In [85]:
predictions = open("predictions_Category.csv", 'w')
pos = 0

for l in open("assignment1/pairs_Category.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    _ = predictions.write(u + ',' + b + ',' + str(pred_test[pos]) + '\n')
    pos += 1

predictions.close()

In [86]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()